Credit cs231n.stanford.edu

## Base Functions 
The `forward` function will receive inputs, weights, and other parameters and will return both an output and a `cache` object storing data needed for the backward pass, like this:

```python
def layer_forward(x, w):
  """ Receive inputs x and weights w """
  # Do some computations ...
  z = # ... some intermediate value
  # Do some more computations ...
  out = # the output
   
  cache = (x, w, z, out) # Values we need to compute gradients
   
  return out, cache
```

The backward pass will receive upstream derivatives and the `cache` object, and will return gradients with respect to the inputs and weights, like this:

```python
def layer_backward(dout, cache):
  """
  Receive derivative of loss with respect to outputs and cache,
  and compute derivative with respect to inputs.
  """
  # Unpack cache values
  x, w, z, out = cache
  
  # Use values in cache to compute derivatives
  dx = # Derivative of loss with respect to x
  dw = # Derivative of loss with respect to w
  
  return dx, dw
```

<h2 align="center">BackProp and Optimizers</h2>
<img src="img/bp.png" width="600">

In [1]:
import numpy as np
from scipy.optimize import check_grad
from gradient_check import eval_numerical_gradient_array

def rel_error(x, y):
      return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

<h3>Grad Check</h3>

<img src="img/gc.png" width="600">

<h3>Softmax Loss Layer</h3>
<img src="./img/loss.png" width="300">
<img src="./img/log.png" width="600">

In [2]:
 def softmax_loss(x, y):
    """
    Computes the loss and gradient for softmax classification.

    Inputs:
    - x: Input data, of shape (N, C) where x[i, j] is the score for the jth class
    for the ith input.
    - y: Vector of labels, of shape (N,) where y[i] is the label for x[i] and
    0 <= y[i] < C

    Returns a tuple of:
    - loss: Scalar giving the loss
    - dx: Gradient of the loss with respect to x
    """
    N = len(y)
    exp_scores = np.exp(x) 
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) 
    loss = np.log(probs[range(N),y])
    loss = -float(np.sum(loss)) / len(y)
    dx = probs
    dx[range(N),y] -= 1
    dx /= N
    return loss, dx

In [3]:
y = np.random.randint(0, 3, 10)
dx = lambda x: softmax_loss(x.reshape((10, 3)), y)[1].reshape(-1)
loss = lambda x: softmax_loss(x.reshape((10, 3)), y)[0]

In [4]:
print 'loss is a scalar\n', loss(np.random.random((10, 3)))

loss is a scalar
1.01391771191


In [5]:
print 'gradient is a matrix with shape 10x3\n', dx(np.random.random((10, 3)))

gradient is a matrix with shape 10x3
[-0.07090429  0.03682753  0.03407676  0.0434172   0.02683517 -0.07025237
 -0.06966497  0.02354675  0.04611822  0.02788587 -0.0683026   0.04041674
 -0.07864474  0.03826714  0.04037759 -0.07687782  0.04187606  0.03500177
  0.03813879 -0.06248451  0.02434572  0.03951789 -0.07920243  0.03968453
  0.04560152  0.03115246 -0.07675398 -0.06944312  0.02834623  0.04109688]


In [6]:
print 'difference should be ~10e-8', check_grad(loss, dx, np.random.random((10, 3)).reshape(-1))

difference should be ~10e-8 8.38181293646e-08


<h3>Dense Layer</h3>
<img src="img/lin.png" width="300">

In [7]:
def affine_forward(x, w, b):
    """
    Computes the forward pass for an affine (fully-connected) layer.

    The input x has shape (N, d_1, ..., d_k) and contains a minibatch of N
    examples, where each example x[i] has shape (d_1, ..., d_k). We will
    reshape each input into a vector of dimension D = d_1 * ... * d_k, and
    then transform it to an output vector of dimension M.

    Inputs:
    - x: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - w: A numpy array of weights, of shape (D, M)
    - b: A numpy array of biases, of shape (M,)

    Returns a tuple of:
    - out: output, of shape (N, M)
    - cache: (x, w, b)
    """
    out = None
    #############################################################################
    # TODO: Implement the affine forward pass. Store the result in out. You     #
    # will need to reshape the input into rows.                                 #
    #############################################################################
    out = np.dot(x.reshape(x.shape[0],np.prod(x.shape[1:])), w) + b
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    cache = (x, w, b)
    return out, cache

In [8]:
# Test the affine_forward function

num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num=input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num=weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num=output_dim)

out, _ = affine_forward(x, w, b)
correct_out = np.array([[ 1.49834967,  1.70660132,  1.91485297],
                        [ 3.25553199,  3.5141327,   3.77273342]])

# Compare your output with ours. The error should be around 1e-9.
print 'Testing affine_forward function:'
print 'difference: ', rel_error(out, correct_out)

Testing affine_forward function:
difference:  9.76985004799e-10


In [9]:
def affine_backward(dout, cache):
    """
    Computes the backward pass for an affine layer.

    Inputs:
    - dout: Upstream derivative, of shape (N, M)
    - cache: Tuple of:
    - x: Input data, of shape (N, d_1, ... d_k)
    - w: Weights, of shape (D, M)

    Returns a tuple of:
    - dx: Gradient with respect to x, of shape (N, d1, ..., d_k)
    - dw: Gradient with respect to w, of shape (D, M)
    - db: Gradient with respect to b, of shape (M,)
    """
    x, w, b = cache
    dx, dw, db = None, None, None
    #############################################################################
    # TODO: Implement the affine backward pass.                                 #
    #############################################################################
    dx = dout.dot(w.T)
    dw = (x.reshape(x.shape[0],np.prod(x.shape[1:]))).T.dot(dout)
    db = np.sum(dout, axis=0)
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    return dx, dw, db

In [10]:
# Test the affine_backward function

x = np.random.randn(10, 2, 3)
w = np.random.randn(6, 5)
b = np.random.randn(5)
dout = np.random.randn(10, 5)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

_, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)
dx_num = dx_num.reshape(dx_num.shape[0], -1)
# The error should be around 1e-10
print 'Testing affine_backward function:'
print 'dx error: ', rel_error(dx_num, dx)
print 'dw error: ', rel_error(dw_num, dw)
print 'db error: ', rel_error(db_num, db)

Testing affine_backward function:
dx error:  4.31043960439e-10
dw error:  1.80120308231e-10
db error:  2.5896892292e-11


<h3>ReLu Layer</h3>

$$ReLu(x) = max(0, x)$$

In [11]:
def relu_forward(x):
    """
    Computes the forward pass for a layer of rectified linear units (ReLUs).

    Input:
    - x: Inputs, of any shape

    Returns a tuple of:
    - out: Output, of the same shape as x
    - cache: x
    """
    out = None
    #############################################################################
    # TODO: Implement the ReLU forward pass.                                    #
    #############################################################################
    out = np.maximum(0, x) 
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    cache = x
    return out, cache

In [12]:
# Test the relu_forward function

x = np.linspace(-0.5, 0.5, num=12).reshape(3, 4)

out, _ = relu_forward(x)
correct_out = np.array([[ 0.,          0.,          0.,          0.,        ],
                        [ 0.,          0.,          0.04545455,  0.13636364,],
                        [ 0.22727273,  0.31818182,  0.40909091,  0.5,       ]])

# Compare your output with ours. The error should be around 1e-8
print 'Testing relu_forward function:'
print 'difference: ', rel_error(out, correct_out)

Testing relu_forward function:
difference:  4.99999979802e-08


In [13]:
def relu_backward(dout, cache):
    """
    Computes the backward pass for a layer of rectified linear units (ReLUs).

    Input:
    - dout: Upstream derivatives, of any shape
    - cache: Input x, of same shape as dout

    Returns:
    - dx: Gradient with respect to x
    """
    dx, x = None, cache
    #############################################################################
    # TODO: Implement the ReLU backward pass.                                   #
    #############################################################################
    dx = dout
    dx[x <= 0]  = 0
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    return dx

In [14]:
x = np.random.randn(10, 10)
dout = np.random.randn(*x.shape)

dx_num = eval_numerical_gradient_array(lambda x: relu_forward(x)[0], x, dout)
_, cache = relu_forward(x)
dx = relu_backward(dout, cache)

# The error should be around 1e-12
print 'Testing relu_backward function:'
print 'dx error: ', rel_error(dx_num, dx)

Testing relu_backward function:
dx error:  3.27560561328e-12


<h3>Two Layer Fully Connected Neural Net with SGD</h3>

In [15]:
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

/Users/taraskhakhulin/Library/Python/2.7/lib/python/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [16]:
std = 1e-4

W1, b1 = std*np.random.random((64, 100)), np.random.random(100)
W2, b2 = std*np.random.random((100, 10)), np.random.random(10)
lr = 1e-4

for i in range(50000):
    batch_index = np.random.randint(0, X_train.shape[0], 100)
    batch_X, batch_y = X_train[batch_index], y_train[batch_index]
    # ------------ Train ----------------- 
    # Forward Pass
    fc_1 = np.dot(X, W1) + b1
    score_1 = np.maximum(0, fc_1)
    scores = np.dot(score_1, W2) + b2

    
    out1, cache1 = affine_forward(batch_X, W1, b1) # Dense Layer
    out2, cache2 = relu_forward(out1)              # ReLu Layer
    out3, cache3 = affine_forward(out2, W2, b2) # Dense Layer 
    tr_loss, dx = softmax_loss(out3, batch_y)      # Loss Layer 
    # Backward Pass
    dlinear2, dw2, db2 = affine_backward(dx, cache3)
    drelu = relu_backward(dlinear2, cache2)
    dlinea1r, dw1, db1 = affine_backward(drelu, cache1)
    W1 = W1 - dw1 * lr
    b1 = b1 - lr*db1
    b2 = b2 - lr*db2
    W2 = W2 - lr*dw2
    # ------------ Test ----------------- 
    # Forward Pass
    scores = np.dot(np.maximum(0, np.dot(X_test, W1)+ b1), W2) + b2
    y_pred = np.argmax(scores, axis=1)
    te_loss,_ = softmax_loss(scores, y_pred)
    # Predict
    if i % 1000 == 0:
        print 'epoch %s:' % i, 
        print '\t tr_loss %.2f' % tr_loss,
        print '\t te_loss %.2f' % te_loss,
        print '\t te_acc %s' % accuracy_score(y_pred, y_test)

epoch 0: 	 tr_loss 2.35 	 te_loss 1.92 	 te_acc 0.0925925925926
epoch 1000: 	 tr_loss 2.32 	 te_loss 2.03 	 te_acc 0.0925925925926
epoch 2000: 	 tr_loss 2.29 	 te_loss 2.13 	 te_acc 0.0925925925926
epoch 3000: 	 tr_loss 2.29 	 te_loss 2.20 	 te_acc 0.0925925925926
epoch 4000: 	 tr_loss 2.28 	 te_loss 2.22 	 te_acc 0.188888888889
epoch 5000: 	 tr_loss 2.27 	 te_loss 2.17 	 te_acc 0.161111111111
epoch 6000: 	 tr_loss 2.25 	 te_loss 2.12 	 te_acc 0.153703703704
epoch 7000: 	 tr_loss 2.23 	 te_loss 2.06 	 te_acc 0.162962962963
epoch 8000: 	 tr_loss 2.19 	 te_loss 1.99 	 te_acc 0.175925925926
epoch 9000: 	 tr_loss 2.11 	 te_loss 1.91 	 te_acc 0.185185185185
epoch 10000: 	 tr_loss 2.09 	 te_loss 1.82 	 te_acc 0.181481481481
epoch 11000: 	 tr_loss 2.05 	 te_loss 1.76 	 te_acc 0.185185185185
epoch 12000: 	 tr_loss 2.01 	 te_loss 1.72 	 te_acc 0.185185185185
epoch 13000: 	 tr_loss 2.07 	 te_loss 1.70 	 te_acc 0.188888888889
epoch 14000: 	 tr_loss 2.03 	 te_loss 1.68 	 te_acc 0.192592592593
epoc